# Set up the environment

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.plot([0,1],[1,0])
plt.show()
import hide_imports
from neuronunit.optimisation.optimization_management import inject_and_plot_model
import copy
import pickle


# Design simulated data tests

In [ ]:
def jrt(use_test,backend):
    use_test = hide_imports.TSD(use_test)
    use_test.use_rheobase_score = True
    edges = hide_imports.model_parameters.MODEL_PARAMS[backend]

    OM = hide_imports.OptMan(use_test,\
                backend=backend,\
                boundary_dict=edges,\
                protocol={'allen': False, 'elephant': True})

    return OM



In [ ]:
backend = ""
MU = ""
NGEN = ""

In [ ]:

test_frame = pickle.load(open('processed_multicellular_constraints.p','rb'))
#test_arg = tests[test_type]
#test_frame = hide_imports.get_neab.process_all_cells()
#test_frame.pop('Olfactory bulb (main) mitral cell',None)
stds = {}
for k,v in hide_imports.TSD(test_frame['Neocortex pyramidal cell layer 5-6']).items():
    temp = hide_imports.TSD(test_frame['Neocortex pyramidal cell layer 5-6'])[k]
    stds[k] = temp.observation['std']
    #print((temp.name,temp.observation))
OMObjects = []
cloned_tests = copy.copy(test_frame['Neocortex pyramidal cell layer 5-6'])
print(backend)
#for b in backends:        
OM = jrt(cloned_tests,backend)
rt_outs = []

In [ ]:
#for OM in OMObjects:
x= {k:v for k,v in OM.tests.items() if 'mean' in v.observation.keys() or 'value' in v.observation.keys()}
cloned_tests = copy.copy(OM.tests)
OM.tests = hide_imports.TSD(cloned_tests)
rt_out = OM.simulate_data(OM.tests,OM.backend,OM.boundary_dict)


In [ ]:
penultimate_tests = hide_imports.TSD(test_frame['Neocortex pyramidal cell layer 5-6'])
for k,v in penultimate_tests.items():
    temp = penultimate_tests[k]

    v = rt_out[1][k].observation
    v['std'] = stds[k]
simulated_data_tests = hide_imports.TSD(penultimate_tests)

# Show what the randomly generated target waveform the optimizer needs to find actually looks like

In [ ]:
target = rt_out[0]
target.rheobase
inject_and_plot_model(target)

# first lets just optimize over all objective functions all the time.
# Commence optimization of models on simulated data sets

In [ ]:
%%capture
import contextlib, io, warnings
warnings.filterwarnings('ignore')
s = io.StringIO()
with contextlib.redirect_stdout(s):
    print('dgrrg')
    warnings.warn('shhth')
s.getvalue()
ga_out = simulated_data_tests.optimize(OM.boundary_dict,backend=OM.backend,\
        protocol={'allen': False, 'elephant': True},\
            MU=MU,NGEN=NGEN)
opt = ga_out['pf'][0].dtc
front = [ind.dtc for ind in ga_out['pf']]

In [ ]:
%%capture
from neuronunit.optimisation.optimization_management import check_match_front
check_match_front(target,front[0:10])



# How about using all the objectives together?


In [ ]:
opt.rheobase


In [ ]:
inject_and_plot_model(opt)

In [ ]:
opt.obs_preds

In [ ]:
model = opt.dtc_to_model()
